In [ ]:
from huggingface_hub import login

login(token="...")

## Local Model

Make sure you have a local model running on localhost:1234/v1 (e.g. useing LM Studio).

In [ ]:
from smolagents import OpenAIServerModel

model = OpenAIServerModel(
    api_base="http://host.docker.internal:1234/v1/",
    model_id="qwen2.5-3b-instruct-mlx",
    api_key="none"
)

In [ ]:
messages = [
    {"role": "system", "content": "You're a helpful assistant."},
    {"role": "user", "content": "Once upon a time, in a land far away,"}
]

response = model(messages)
print(response.content)

## Research the Internet

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

## Use a Custom Tool

In [ ]:
from smolagents import tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.

    Args:
        occasion (str): The type of occasion for the party. Must be one of 'casual', 'formal', 'superhero', or 'custom'.

    Returns:
        str: A suggested menu tailored to the specified occasion.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

In [ ]:
from smolagents import CodeAgent

# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(tools=[suggest_menu], model=model)

# Preparing the menu for the party
agent.run("Prepare a formal menu for the party.")

## Allow Python Imports

In [ ]:
from smolagents import CodeAgent

agent = CodeAgent(tools=[], model=model, additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

## Push and Pull Agent - HuggingFace

In [ ]:
# requires a token with create spaces rights

# agent.push_to_hub('almightyt/name-of-the-agent')


In [ ]:
# agent = agent.from_hub('sergiopaniego/name-of-the-agent', trust_remote_code=True)
# agent.run("Give me the best playlist for a party at Wayne's mansion. The party idea is a 'villain masquerade' theme")  


## Combining Ideas

In [ ]:
from smolagents import tool
import sqlite3


@tool
def add_gotcha_note(error_message: str, solution: str) -> str:
    """
    Adds a gotcha note to the database.

    Args:
        error_message: Error and short description.
        solution: Explanation of how the error was resolved. This should only include the final solution, to keep it brief.

    Returns:
        str: Confirmation message upon successful addition.

    Example:
        add_gotcha_note("IndexError: list index out of range", "Ensured index is within list length.")
    """
    conn = sqlite3.connect('gotchas.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO gotchas (error_message, solution) VALUES (?, ?)', (error_message, solution))
    conn.commit()
    conn.close()
    return "Gotcha note added successfully."


@tool
def get_all_gotchas_notes() -> str:
    """
    Retrieves all gotcha notes from the database.

    Returns:
        str: Formatted string of all recorded gotchas.

    Example:
        get_all_gotchas_notes()
    """
    conn = sqlite3.connect('gotchas.db')
    cursor = conn.cursor()
    cursor.execute('SELECT error_message, solution, timestamp FROM gotchas ORDER BY timestamp DESC')
    rows = cursor.fetchall()
    conn.close()
    if not rows:
        return "No gotcha notes found."
    notes = "\n".join([f"[{timestamp}] Error: {error}\nSolution: {solution}\n" for error, solution, timestamp in rows])
    return notes


In [ ]:
@tool
def summairze_text(text: str) -> str:
    """
    Summarizes the text.

    Args:
        text: The text to be summarized.

    Returns:
        str: Summarized text.
    """
    messages = [
        {"role": "system", "content": "You are an expert summarizer. You summarize text that is given to you as succinctly as possible, keeping only the most important information."},
        {"role": "user", "content": f"Summarize the following text: {text}"}
    ]

    return model(messages).content

In [ ]:
summairze_text("""The University of Oslo offers PhD programs in various fields, including the 
humanities, law, medicine and health sciences, dentistry, mathematics and natural sciences, social science, and 
theology. Admissions require completion of a Master’s degree and meet financial requirements. The university 
provides comprehensive information on programs, including details on application processes, structures, and 
thesis/public defenses. Notable public defense events are listed, such as Rasmus Nilsson's on March 14 at the 
Medical Faculty. \n\nThe university also awards the Dr.Philos. degree to academics who complete their doctoral exam
independently, without formal supervision. Students interested in these programs can contact the university 
directly via various channels including their website, emails, and administrative interfaces.""")

In [ ]:
from smolagents.tools import Tool
import requests
from markdownify import markdownify
import re

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    inputs = {'url': {'type': 'string', 'description': 'The url of the webpage to visit.'}}
    output_type = "string"

    def forward(self, url: str) -> str:
        try:
            from requests.exceptions import RequestException
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            # Send a GET request to the URL with a 20-second timeout
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes

            # Convert the HTML content to Markdown
            markdown_content = markdownify(response.text).strip()

            # Remove multiple line breaks
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

            return summairze_text(markdown_content)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

    def __init__(self, *args, **kwargs):
        self.is_initialized = False


visit_webpage = VisitWebpageTool()

In [ ]:
# add_gotcha_note("ImportError: datetime is not allowed", "Ask for adjustment of agent code.")

In [ ]:
get_all_gotchas_notes()

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool

system_prompt = """You are an AI internet research assistant specialized in researching specific questions and completing tasks related to internet search.
You are an expert at avoiding mistakes you have already made; whenever an error occurs, you resolve it intelligently, and then you log the initial error and the final solution using the `add_gotcha_note` tool.
Whenever a new task is asked of you, you first use the `get_all_gotchas_notes` tool to remember all the gotchas or pitfalls to avoid.
When you encounter an error, search for it using the `search_gotcha_notes` tool to see if you already encountered it before and already know how to solve it."""

# prompt_templates = {
#     "system_prompt": system_prompt,
#     "planning": {
#         "initial_facts": "Gather initial requirements and constraints.",
#         "initial_plan": "Develop an initial plan based on the facts.",
#         "update_facts_pre_messages": "Before updating facts, consider the following messages.",
#         "update_facts_post_messages": "After updating facts, note the following.",
#         "update_plan_pre_messages": "Before updating the plan, consider these messages.",
#         "update_plan_post_messages": "After updating the plan, note the following."
#     },
#     "managed_agent": {
#         "task": "Execute the following development task: {task}",
#         "report": "Provide a report on the completed task."
#     },
#     "final_answer": {
#         "pre_messages": "Before finalizing, consider these points.",
#         "post_messages": "After finalizing, note the following."
#     }
# }

agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), add_gotcha_note, get_all_gotchas_notes, search_gotcha_notes, visit_webpage],
    model=model,
    # prompt_templates=prompt_templates,
    additional_authorized_imports=['datetime', 'pandas', 'numpy', 'requests', 'json', 're', 'math'],
)

### Change the System Prompt

In [ ]:
initial_system_prompt = agent.system_prompt
# print(initial_system_prompt)

system_prompt = """You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with the '<end_code>' sequence.
During each intermediate step, you should use 'print()' to save important information you will need later: These print outputs will appear in the 'Observation:' field, which will be available as input for the next step, so they're quite important. At the same time, you should only print what is really needed for future steps.
You are also great at solving problems, as well as remembering and avoiding previous errors. Therefore, you must use `get_all_gotchas_notes` to look up learnings from previous tasks whenever you start a new task.
In the end, you have to return a final answer using the `final_answer` tool.

Here are a few examples using notional tools:
---
Task: "Generate an image of the oldest person in this document."

Thought: First, I will use the tool `get_all_gotchas_notes` to ensure the resolution of previous error is in my short-term memory. I will print them to make sure I get them as response to my tool call.
Code:
```py
previous_gotchas = get_all_gotchas_notes()
print(previous_gotchas)
```<end_code>
Observation: "[2025-03-12 21:17:11] Error: ImportError: datetime is not allowed\nSolution: Ask for adjustment of agent code.\n"

Thought: I will proceed step by step and use the following tools: `document_qa` to find the oldest person in the document, then `image_generator` to generate an image according to the answer.
Code:
```py
answer = document_qa(document=document, question="Who is the oldest person mentioned?")
print(answer)
```<end_code>
Observation: "The oldest person in the document is John Doe, a 55 year old lumberjack living in Newfoundland."

Thought: I will now generate an image showcasing the oldest person.
Code:
```py
image = image_generator("A portrait of John Doe, a 55-year-old man living in Canada.")
final_answer(image)
```<end_code>

---
Task: "What is the result of the following operation: 5 + 3 + 1294.678?"

Thought: First, I will use the tool `get_all_gotchas_notes` to ensure the resolution of previous error is in my short-term memory. I will print them to make sure I get them as response to my tool call.
Code:
```py
previous_gotchas = get_all_gotchas_notes()
print(previous_gotchas)
```<end_code>
Observation: "[2025-03-12 21:17:11] Error: ImportError: datetime is not allowed\nSolution: Ask for adjustment of agent code.\n"

Thought: I will use python code to compute the result of the operation and then return the final answer using the `final_answer` tool
Code:
```py
result = 5 + 3 + 1294.678
final_answer(result)
```<end_code>

---
Task:
"Answer the question in the variable `question` about the image stored in the variable `image`. The question is in French.
You have been provided with these additional arguments, that you can access using the keys as variables in your python code:
{'question': 'Quel est l'animal sur l'image?', 'image': 'path/to/image.jpg'}"

Thought: First, I will use the tool `get_all_gotchas_notes` to ensure the resolution of previous error is in my short-term memory. I will print them to make sure I get them as response to my tool call.
Code:
```py
previous_gotchas = get_all_gotchas_notes()
print(previous_gotchas)
```<end_code>
Observation: "[2025-03-12 21:17:11] Error: ImportError: datetime is not allowed\nSolution: Ask for adjustment of agent code.\n"

Thought: I will use the following tools: `translator` to translate the question into English and then `image_qa` to answer the question on the input image.
Code:
```py
translated_question = translator(question=question, src_lang="French", tgt_lang="English")
print(f"The translated question is {translated_question}.")
answer = image_qa(image=image, question=translated_question)
final_answer(f"The answer is {answer}")
```<end_code>

---
Task:
In a 1979 interview, Stanislaus Ulam discusses with Martin Sherwin about other great physicists of his time, including Oppenheimer.
What does he say was the consequence of Einstein learning too much math on his creativity, in one word?

Thought: First, I will use the tool `get_all_gotchas_notes` to ensure the resolution of previous error is in my short-term memory. I will print them to make sure I get them as response to my tool call.
Code:
```py
previous_gotchas = get_all_gotchas_notes()
print(previous_gotchas)
```<end_code>
Observation: "[2025-03-12 21:17:11] Error: ImportError: datetime is not allowed\nSolution: Ask for adjustment of agent code.\n"

Thought: I need to find and read the 1979 interview of Stanislaus Ulam with Martin Sherwin.
Code:
```py
pages = search(query="1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein")
print(pages)
```<end_code>
Observation:
No result found for query "1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein".

Thought: The query was maybe too restrictive and did not find any results. Let's try again with a broader query.
Code:
```py
pages = search(query="1979 interview Stanislaus Ulam")
print(pages)
```<end_code>
Observation:
Found 6 pages:
[Stanislaus Ulam 1979 interview](https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/)

[Ulam discusses Manhattan Project](https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/)

(truncated)

Thought: I will read the first 2 pages to know more.
Code:
```py
for url in ["https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/", "https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/"]:
    whole_page = visit_webpage(url)
    print(whole_page)
    print("\n" + "="*80 + "\n")  # Print separator between pages
```<end_code>
Observation:
Manhattan Project Locations:
Los Alamos, NM
Stanislaus Ulam was a Polish-American mathematician. He worked on the Manhattan Project at Los Alamos and later helped design the hydrogen bomb. In this interview, he discusses his work at
(truncated)

Thought: I now have the final answer: from the webpages visited, Stanislaus Ulam says of Einstein: "He learned too much mathematics and sort of diminished, it seems to me personally, it seems to me his purely physics creativity." Let's answer in one word.
Code:
```py
final_answer("diminished")
```<end_code>

---
Task: "Which city has the highest population: Guangzhou or Shanghai?"

Thought: First, I will use the tool `get_all_gotchas_notes` to ensure the resolution of previous error is in my short-term memory. I will print them to make sure I get them as response to my tool call.
Code:
```py
previous_gotchas = get_all_gotchas_notes()
print(previous_gotchas)
```<end_code>
Observation: "[2025-03-12 21:17:11] Error: ImportError: datetime is not allowed\nSolution: Ask for adjustment of agent code.\n"

Thought: I need to get the populations for both cities and compare them: I will use the tool `search` to get the population of both cities.
Code:
```py
for city in ["Guangzhou", "Shanghai"]:
    print(f"Population {city}:", search(f"{city} population")
```<end_code>
Observation:
Population Guangzhou: ['Guangzhou has a population of 15 million inhabitants as of 2021.']
Population Shanghai: '26 million (2019)'

Thought: Now I know that Shanghai has the highest population.
Code:
```py
final_answer("Shanghai")
```<end_code>

---
Task: "What is the current age of the pope, raised to the power 0.36?"

Thought: First, I will use the tool `get_all_gotchas_notes` to ensure the resolution of previous error is in my short-term memory. I will print them to make sure I get them as response to my tool call.
Code:
```py
previous_gotchas = get_all_gotchas_notes()
print(previous_gotchas)
```<end_code>
Observation: "[2025-03-12 21:17:11] Error: ImportError: datetime is not allowed\nSolution: Ask for adjustment of agent code.\n"

Thought: I will use the tool `wiki` to get the age of the pope, and confirm that with a web search.
Code:
```py
pope_age_wiki = wiki(query="current pope age")
print("Pope age as per wikipedia:", pope_age_wiki)
pope_age_search = web_search(query="current pope age")
print("Pope age as per google search:", pope_age_search)
```<end_code>
Observation:
Pope age: "The pope Francis is currently 88 years old."

Thought: I know that the pope is 88 years old. Let's compute the result using python code.
Code:
```py
pope_current_age = 88 ** 0.36
final_answer(pope_current_age)
```<end_code>

---

Above examples were using notional tools that might not exist for you. On top of performing computations in the Python code snippets that you create, you only have access to these tools:
- web_search: Performs a Google web search based on your query, then returns the top search results.
    Takes inputs: {'query': {'type': 'string', 'description': 'The search query to perform.'}}
    Returns an output of type: string
- visit_webpage: Visits a specific webpage at the given url and reads its content as a markdown string. Use this to browse webpages and their contents in detail.
    Takes inputs: {'url': {'type': 'string', 'description': 'The url of the webpage to visit.'}}
    Returns an output of type: string
- get_all_gotchas_notes: Retrieves all gotcha notes from the database.
    Takes inputs: {}
    Returns an output of type: string
- add_gotcha_note: Adds a gotcha note to the database.
    Takes inputs: {'error_message': {'type': 'string', 'description': 'Error and short description.'}, 'solution': {'type': 'string', 'description': 'Explanation of how the error was resolved. This should only include the final solution, to keep it brief.'}}
    Returns an output of type: string
- final_answer: Provides a final answer to the given problem.
    Takes inputs: {'answer': {'type': 'any', 'description': 'The final answer to the problem'}}
    Returns an output of type: any

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a 'Code:\n```py' sequence ending with '```<end_code>' sequence, else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wiki({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wiki(query="What is the place where James Bond lives?")'.
4. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
5. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
6. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
7. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
8. You can use imports in your code, but only from the following list of modules: ['stat', 'itertools', 'unicodedata', 'random', 'datetime', 'collections', 'json', 're', 'queue', 'statistics', 'time', 'requests', 'math', 'pandas', 'numpy']
9. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
10. Don't give up! You're in charge of solving the task, not providing directions to solve it.
11. When you start a task, use `get_all_gotchas_notes` to look up learning from previous tasks. When you encounter an error, solve it intelligently, and then record your findings using `add_gotcha_note`.

Now Begin! If you solve the task correctly, you will receive a reward of $1,000,000.
"""

agent.system_prompt = system_prompt

In [ ]:
agent.run(
"""
I'm looking for PhD fellowship or doctoral candidate positions in IT/Computer Science/AI. I prefer applied research.
I want it to be a PhD program that takes 3 years to complete at a university or ideally a university college/university of applied sciences.
I heard a duration of 3 years to complete a PhD program is common in some countries, like Norway.
The program should not include any teaching or other responsibilities besides research (and perhaps a few courses).
The program must be paid, or be part-time.
The program should be open for application at the moment.
List at least 5 options you can find, including university name, location (city, country), field of study, duration, salary, and official URL to the program of the organization that hosts it.
"""
)